In [48]:
import warnings
warnings.filterwarnings('ignore')
# import geoplot as gplt

# import geopandas as gpd
# import geoplot.crs as gcrs
import imageio
import pandas as pd
import pathlib
import matplotlib.pyplot as plt
import mapclassify as mc
import numpy as np
import laspy
import rasterio
from rasterio import mask
import folium

In [14]:
# !pip install geopandas

In [13]:
# import geopandas

In [53]:
import json, ast
from os import stat
import pdal


def read_json(file_name):
  '''
  Read json file and return the string format
  '''

  try:
    file_path = f"../{file_name}"
    print("File Path : ", file_path)
    with open(file_path, 'r') as json_file:
      data = json.loads(json_file.read())
    return data

  except:
    print('File Not found')
    return None


def prepare_pipe(bound, us_state='IA_FullState'):
    data = read_json('pipeline.json')
    data['pipeline'][0]['bounds'] = bound
    data['pipeline'][0]['filename'] = "https://s3-us-west-2.amazonaws.com/usgs-lidar-public/"+us_state+"/ept.json"
    
    data['pipeline'][6]['filename'] = '../data/laz/'+us_state+'.laz'
    data['pipeline'][7]['filename'] = '../data/tiff/'+us_state+'.tiff'
    

    print("data LInk : " , data['pipeline'][0]['filename'])
    return data


def run_pipe(bound, us_state):
    print("Run pipe ...")
    result = prepare_pipe(bound, us_state)
    pipeline = pdal.Pipeline(json.dumps(result))
    pipe_result  = pipeline.execute()
    print("Fetching Completed!")

def test_json_reading():
    data = read_json('pipeline.json')
    print(data)
    


def test_prepare_pipe(bound):
    result = prepare_pipe(bound)
    print(result)

## Initial Files
bound = str(([-10425171.940, -10423171.940], [5164494.710, 5166494.710]))
state = 'IA_FullState'


## Test Things are working
# test_json_reading()
# test_prepare_pipe(bound)
    

run_pipe(bound, state)


Run pipe ...
File Path :  ../pipeline.json
data LInk :  https://s3-us-west-2.amazonaws.com/usgs-lidar-public/IA_FullState/ept.json
Fetching Completed!


(filters.reprojection Error) GDAL failure (1) PROJ: Unrecognized horizontal grid format
(filters.reprojection Error) GDAL failure (1) PROJ: hgridshift: could not find required grid(s).
(filters.reprojection Error) GDAL failure (1) PROJ: pipeline: Pipeline: Bad step definition: inv (File not found or invalid)
(filters.reprojection Error) GDAL failure (1) PROJ: Unrecognized horizontal grid format
(filters.reprojection Error) GDAL failure (1) PROJ: hgridshift: could not find required grid(s).
(filters.reprojection Error) GDAL failure (1) PROJ: pipeline: Pipeline: Bad step definition: inv (File not found or invalid)
(filters.reprojection Error) GDAL failure (1) PROJ: Unrecognized horizontal grid format
(filters.reprojection Error) GDAL failure (1) PROJ: hgridshift: could not find required grid(s).
(filters.reprojection Error) GDAL failure (1) PROJ: pipeline: Pipeline: Bad step definition: inv (File not found or invalid)
(filters.reprojection Error) GDAL failure (1) PROJ: Unrecognized horiz

In [54]:
import pdal

In [32]:
import laspy

In [4]:
import geopandas as gpd

In [ ]:
## Read file with laspy the .laz file
## Generate POINT with x, y
## Elevation with Z
## Geopandas Data frame, two columns -> Elevation : Z, Geometry: POINTS
## Return the Dataframe

In [40]:
def read_laz(file_name):
    try:
        las = laspy.read(file_name)
        return las
    except FileNotFoundError:
        print("Log: File Not found")
las_path = '../data/las/IA_FullState.las'    
las_file = read_laz(las_path)

In [39]:
from shapely.geometry import box, Point, Polygon

In [41]:
## Computation Intensive
def generate_points_elevation(las_file):
    '''
    Return Points (x, y) and elevation (z)
    '''
    points = [Point(x, y) for x,y in zip(las_file.x, las_file.y)]
    elevation = np.array(las_file.z)
    
    return points, elevation


    
points, elevation = generate_points_elevation(las_file)

In [44]:
points[:10]

In [43]:
elevation[:10]

array([275.72, 275.75, 274.39, 275.89, 276.1 , 276.17, 276.4 , 276.81,
       276.54, 276.1 ])

In [45]:
import geopandas as gpd

In [47]:
def create_geopandasdf(geometry_points, elevation):
    '''
    Return Geopandas data frame from elevation and geometic points
    '''
    geo_df = gpd.GeoDataFrame({"elevation": elevation, "geometry":geometry_points})
    geo_df.set_geometry('geometry')
    geo_df.set_crs(epsg=26915, inplace=True)

    return geo_df

df = create_geopandasdf(points, elevation)
df.head()


,elevation,geometry
0,275.72,POINT (447494.080 4653790.160)
1,275.75,POINT (447488.810 4653790.160)
2,274.39,POINT (447476.510 4653788.410)
3,275.89,POINT (447485.510 4653790.420)
4,276.10,POINT (447489.140 4653791.210)
